In [1]:
from itertools import combinations, product, chain
import os
import numpy as np

In [ ]:
print(os.getcwd()) 

In [2]:
#Loading Golay Code
codewords_weight_8  = np.load("data_golay-code/codewords_weight_8.npy")
codewords_weight_12  = np.load("data_golay-code/codewords_weight_12.npy")
codewords_weight_16  = np.load("data_golay-code/codewords_weight_16.npy")
C  = np.load("data_golay-code/C.npy")

In [3]:
#Adapted to numpy
def leech_latt_controll_vector(vec):
    v_m = vec % 2
    i = np.all(v_m == v_m[0])
    iii = (np.sum(vec) % 8 == 4*(v_m[0]) % 8)
    norm_sq = np.sum(vec**2)
    b_0 = (vec % 4 == 0) in C
    b_1 = (vec % 4 == 1) in C
    b_2 = (vec % 4 == 2) in C
    b_3 = (vec % 4 == 3) in C
    #actual conditions
    if (norm_sq) != 64: #8*8
        return False
    if not (i and iii and all([b_0, b_1, b_2, b_3])):
        return False
    return True

------------------------------------------------------------------------------------------------------------------------------

Shape $\left( \mp 3^5, (\pm 1)^{19}\right)$, where the entries congruent to $1$ modulo $4$ occupy a $\mathcal{G}$-set.

In [5]:
#Shape ((-+3)^5, (±1)^19), where the entries congruent 1 modulo 4 occupy a G set; comb_oz means on zeros (oz = on zeros)
def shape2(cw):
    vectors = []
    vec = np.zeros(24)
    pos= np.where(cw == 1)[0]
    pos_without_pos = np.setdiff1d(np.arange(24),pos)
    for i in [0,1,2,3,4,5]:
        for comb in combinations(pos,i):
            pos_without_comb = np.setdiff1d(pos, comb)
            for comb_oz in combinations(pos_without_pos,5-i):
                pos_without_pos_without_comb_oz = np.setdiff1d(pos_without_pos,comb_oz)
                vec.fill(0)
                vec[list(comb)] = -3
                vec[list(pos_without_comb)] = 1
                vec[list(comb_oz)] = 3
                vec[list(pos_without_pos_without_comb_oz)] = -1
                vectors.append(vec.copy())
    return vectors

-----------------------------------------------------------------------------------------------------------------------------

**Possibly critical runtime**

In [6]:
M_shape2 = np.apply_along_axis(shape2, axis=1, arr=C)
M_shape2.shape

(4096, 42504, 24)

In [7]:
np.save("shell3_shape2_around_zero", M_shape2)

In [8]:
check = np.apply_along_axis(leech_latt_controll_vector, axis=2, arr=M_shape2)
print(np.all(check))

True


In [ ]:
if np.all(check):
    np.save("shell3_shape2_around_zero", M_shape2)
    print("Daten gespeichert")
else:
    print("Die Lattice Points sind fehlerhaft!")